# 🎵 Obtener Canciones Guardadas de Spotify

Este notebook combina el flujo de autenticación de Spotify con la obtención de canciones guardadas en un solo lugar.

**Pasos:**
1. Generar una URL de autorización.
2. Intercambiar el código de autorización por un token de acceso.
3. Usar el token para obtener las canciones guardadas del usuario.

## Paso 1: Cargar Credenciales y Generar URL de Autorización

In [ ]:
import os
import httpx
import base64
from dotenv import load_dotenv
from urllib.parse import urlencode

# Cargar variables desde el archivo .env
dotenv_path = os.path.join(os.getcwd(), '../../.env')
load_dotenv(dotenv_path=dotenv_path)

client_id = os.environ.get("SPOTIFY_CLIENT_ID")
client_secret = os.environ.get("SPOTIFY_CLIENT_SECRET")
redirect_uri = os.environ.get("SPOTIFY_REDIRECT_URI")

if not all([client_id, client_secret, redirect_uri]):
    print("❌ Faltan una o más variables de Spotify (SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET, SPOTIFY_REDIRECT_URI) en tu .env")
else:
    print("✅ Credenciales de Spotify cargadas.")
    
    # Definir el scope (qué permisos solicitamos)
    scope = 'user-library-read'
    
    # Construir la URL de autorización
    auth_params = {
        'client_id': client_id,
        'response_type': 'code',
        'redirect_uri': redirect_uri,
        'scope': scope,
    }
    auth_url = f"https://accounts.spotify.com/authorize?{urlencode(auth_params)}"
    
    print("PASO SIGUIENTE: Copia la siguiente URL, pégala en tu navegador y autoriza la aplicación.")
    print("Después de autorizar, serás redirigido a una URL que podría dar un error (es normal). Copia el valor del parámetro 'code' de esa URL.")
    print("Ejemplo: http://127.0.0.1:8000/callback?code=AQUI_ESTA_EL_CODIGO_QUE_NECESITAS")
    print("---")
    print(f"{auth_url}")
    print("---")

## Paso 2: Intercambiar el Código de Autorización por un Token de Acceso

Pega el código que obtuviste del navegador en la siguiente celda y ejecútala.

In [ ]:
# Pega aquí el código que copiaste del navegador
auth_code = ""

access_token = None
if not auth_code:
    print("No se ha introducido ningún código. Pega el código de autorización en la variable 'auth_code' y vuelve a ejecutar.")
else:
    print("Código recibido. Solicitando token de acceso...")
    
    token_url = 'https://accounts.spotify.com/api/token'
    
    # Spotify requiere que el client_id y client_secret estén en la cabecera Authorization, codificados en Base64
    auth_header = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    
    headers = {
        'Authorization': f'Basic {auth_header}',
        'Content-Type': 'application/x-www-form-urlencoded',
    }
    
    data = {
        'grant_type': 'authorization_code',
        'code': auth_code,
        'redirect_uri': redirect_uri,
    }
    
    try:
        with httpx.Client() as client:
            response = client.post(token_url, headers=headers, data=data)
            response.raise_for_status() # Lanza una excepción si la respuesta es un error HTTP (4xx o 5xx)
            
            token_data = response.json()
            access_token = token_data.get('access_token')
            
            if access_token:
                print("✅ ¡Autenticación completada exitosamente!")
                print(f"Access Token (primeros 15 caracteres): {access_token[:15]}...")
            else:
                print("❌ No se pudo obtener el token de acceso.")
            
    except httpx.HTTPStatusError as e:
        print(f"❌ Error en la solicitud HTTP: {e}")
        print(f"Respuesta del servidor: {e.response.text}")
        print("Verifica que el 'auth_code' sea correcto y no haya expirado, y que el REDIRECT_URI en tu .env coincida con el configurado en Spotify Developer Dashboard.")
    except Exception as e:
        print(f"❌ Ocurrió un error inesperado: {e}")

## Paso 3: Realizar la Petición para Obtener Canciones Guardadas

Con el token de acceso, construiremos la petición al endpoint `v1/me/tracks`.

In [ ]:
if access_token:
    BASE_URL = 'https://api.spotify.com/v1/me/tracks'
    
    # Parámetros de la petición
    params = {
        'limit': 10,
        'offset': 10,
        'market': 'ES'
    }
    
    # Cabeceras con el token de autorización
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    
    # Realizar la petición GET
    response = httpx.get(BASE_URL, headers=headers, params=params)
    
    # Verificar si la petición fue exitosa
    if response.status_code == 200:
        print("✅ Petición de canciones exitosa! (Código 200)")
        saved_tracks_data = response.json()
    else:
        print(f"❌ Error en la petición de canciones: {response.status_code}")
        print(response.json())
else:
    print("No hay token de acceso disponible. Completa los pasos anteriores.")

## Paso 4: Analizar la Respuesta

In [ ]:
if 'saved_tracks_data' in locals():
    # Imprimir información de paginación
    print(f"Total de canciones guardadas: {saved_tracks_data['total']}")
    print(f"Límite por página: {saved_tracks_data['limit']}")
    print(f"Offset actual: {saved_tracks_data['offset']}")
    print(f"URL siguiente página: {saved_tracks_data['next']}")
    print("---")
    
    # Extraer y mostrar las canciones
    tracks = saved_tracks_data['items']
    
    if tracks:
        print(f"Mostrando {len(tracks)} canciones (del índice {saved_tracks_data['offset'] + 1} al {saved_tracks_data['offset'] + len(tracks)}):")
        for i, item in enumerate(tracks):
            track = item['track']
            artist_names = ', '.join([artist['name'] for artist in track['artists']])
            print(f"{saved_tracks_data['offset'] + i + 1}. {track['name']} - {artist_names}")
    else:
        print("No se encontraron canciones en este rango (offset/limit). Intenta con otros valores.")